In [1]:
import numpy as np
from matplotlib import pyplot as plt
from lmfit import Model
from scipy import integrate
from scipy import optimize
from scipy.optimize import minimize
legend_properties = {'weight':'bold'}
%matplotlib qt

In [2]:
E_0 = (225.0-125.0)*1.0e-06  #convert KeV to GeV
E_0_quad = (225.0e-06)**2 - (125.0e-06)**2
H_0 = 67.3/(3.086e+19) # convert km/sec/Mpc to /sec
O_M = 0.308
O_L = 1 - O_M

In [3]:
data_path = "/media/vyaas/Expansion/LIV_spectral_lag/data/GRB_WeiWu2017.txt"
data = np.loadtxt(data_path, usecols=(1,2,3,4))

In [4]:
data

array([[ 1.44000e+00, -1.36600e+01,  1.84880e+02,  2.18760e+02],
       [ 2.90000e+00,  2.85190e+02,  5.90500e+01,  5.91400e+01],
       [ 6.10000e-01,  5.47200e+01,  2.54200e+01,  2.55900e+01],
       [ 1.71000e+00,  5.55800e+02,  3.86110e+02,  3.95900e+02],
       [ 2.20000e+00,  1.62520e+02,  7.47400e+01,  7.95000e+01],
       [ 4.05000e+00,  2.52400e+02,  8.56500e+01,  8.81800e+01],
       [ 3.91000e+00,  3.49990e+02,  2.33640e+02,  2.37120e+02],
       [ 1.55000e+00,  4.25600e+01,  5.11700e+01,  5.37300e+01],
       [ 1.92000e+00, -1.00010e+02,  1.38040e+02,  1.38730e+02],
       [ 1.88000e+00,  2.30040e+02,  1.69950e+02,  1.75420e+02],
       [ 9.40000e-01, -7.09000e+00,  8.25800e+01,  8.34900e+01],
       [ 5.47000e+00,  1.42600e+01,  1.11900e+02,  1.11690e+02],
       [ 1.26000e+00,  2.70500e+01,  2.54200e+01,  2.68800e+01],
       [ 3.50000e-01, -6.03940e+02,  4.16220e+02,  4.03940e+02],
       [ 1.31000e+00,  2.83600e+01,  2.00200e+01,  2.02500e+01],
       [ 2.09000e+00,  6.

In [5]:
z = data[:, 0]
del_t_obs = data[:, 1]*1.0e-03
sigma_l = data[:, 2]*1.0e-03
sigma_r = data[:, 3]*1.0e-03

In [6]:
sigma_del_t = (sigma_l + sigma_r)/2

In [7]:
rescaled_del_t_obs = del_t_obs /(1+z)

In [8]:
n=1

def integrand_linear(i):
    return ((1+i)**n/np.sqrt((O_M*(1+i)**3)+O_L))

In [9]:
m=2

def integrand_quad(i):
    return ((1+i)**m/np.sqrt((O_M*(1+i)**3)+O_L))

In [10]:
k_linear = []
k_quad = []

for _z in z:
    I1, error = integrate.quad(integrand_linear, 0, _z)
    I2, error = integrate.quad(integrand_quad, 0, _z)
    value_lin = (1/(1+_z)**2)*I1
    value_quad = (1/(1+_z)**3)*I2
    k_linear.append(value_lin)
    k_quad.append(value_quad)
    
K_linear = np.array(k_linear)
K_quad = np.array(k_quad)

In [11]:
K_linear, K_quad

(array([0.26857874, 0.19923748, 0.25862883, 0.25653421, 0.23210355,
        0.15682452, 0.16124821, 0.26394386, 0.24617459, 0.2481775 ,
        0.2781637 , 0.12090287, 0.27482188, 0.20569902, 0.27329968,
        0.23761832, 0.26610924, 0.27195398, 0.23460466, 0.27264007,
        0.2781637 , 0.24466776, 0.27816009, 0.27243801, 0.20857841,
        0.2598509 , 0.19328979, 0.21319707, 0.23210355, 0.26679943,
        0.21696652, 0.23711533, 0.22911906, 0.20496155, 0.26394386,
        0.23711533, 0.24891094, 0.25214992, 0.19496605, 0.21983451,
        0.27537534, 0.27849097, 0.25653421, 0.2402009 , 0.27853271,
        0.27013288, 0.24891094, 0.23661259, 0.23110647, 0.26123952,
        0.25046606, 0.27784683, 0.27744585, 0.2691813 , 0.27361905,
        0.20875175]),
 array([0.18966883, 0.12240542, 0.2106734 , 0.17530155, 0.15086932,
        0.08996389, 0.09316782, 0.18381398, 0.16443611, 0.16646657,
        0.21192796, 0.0653026 , 0.19893267, 0.17953352, 0.19642765,
        0.15605668, 0.1864

In [12]:
def chi2_n0(theta, E, err):
    t, a = theta
    model = t*(E**a - E_0**a)
    term = (t_obs-model)/err
    return np.sum(term ** 2)

In [13]:
def chi2_n1(theta, E_qg, E, err):                                                                                               
    t, a  = theta                                                                                                    
    model = t*(E**a - E_0**a) + (-(1+n)/(2*H_0*3.24))*(E**n-E_0**n)*I1*(10**14)/(E_qg**n)                                          
    term = (t_obs-model)/err                                                                                              
    return np.sum(term ** 2)

In [14]:
def chi2_wei(theta, E_qg, del_T, z, sigma_del_T, k):
    b, sigma_int = theta
    a_liv = E_0/(1.0*H_0*E_qg)
    model = a_liv*k + b
    term_num = ((del_T/(1+z)) - (model))**2
    term_den = (sigma_int**2) + ((sigma_del_T/(1+z))**2)
    term = term_num/term_den
    return np.sum(term)

In [15]:
def chi2_from_likelihood(theta, E_qg, del_T, z, sigma_del_T, k):
    b, sigma_int = theta
    a_liv = E_0/(1.0*H_0*E_qg)
    model = a_liv*k + b
    term_num = ((del_T/(1+z)) - (model))**2
    term_den = (sigma_int**2) + ((sigma_del_T/(1+z))**2)
    term1 = -2*np.log(1/np.sqrt(sigma_int**2 + ((sigma_del_T/(1+z))**2)))
    term2 = term_num/term_den
    return np.sum(term1 + term2)

In [16]:
def chi2_from_likelihood_quad(theta, E_qg, del_T, z, sigma_del_T, k):
    b, sigma_int = theta
    a_liv = (3.0*E_0_quad)/(2.0*H_0*(E_qg**2))
    model = a_liv*k + b
    term_num = ((del_T/(1+z)) - (model))**2
    term_den = (sigma_int**2) + ((sigma_del_T/(1+z))**2)
    term1 = -2*np.log(1/np.sqrt(sigma_int**2 + ((sigma_del_T/(1+z))**2)))
    term2 = term_num/term_den
    return np.sum(term1 + term2)

In [17]:
x = np.logspace(5,19,25)
print(x)

[1.00000000e+05 3.83118685e+05 1.46779927e+06 5.62341325e+06
 2.15443469e+07 8.25404185e+07 3.16227766e+08 1.21152766e+09
 4.64158883e+09 1.77827941e+10 6.81292069e+10 2.61015722e+11
 1.00000000e+12 3.83118685e+12 1.46779927e+13 5.62341325e+13
 2.15443469e+14 8.25404185e+14 3.16227766e+15 1.21152766e+16
 4.64158883e+16 1.77827941e+17 6.81292069e+17 2.61015722e+18
 1.00000000e+19]


In [18]:
theta_guess = [0.03, 0.03]
params_linear = [optimize.fmin(chi2_from_likelihood, theta_guess, args=(E_QG, del_t_obs, z, sigma_del_t, K_linear), maxfun=5000) for E_QG in x]
print("Length of params n=1 : ", len(params_linear))

Optimization terminated successfully.
         Current function value: 1910.723734
         Iterations: 206
         Function evaluations: 418
Optimization terminated successfully.
         Current function value: 1760.288174
         Iterations: 190
         Function evaluations: 382
Optimization terminated successfully.
         Current function value: 1609.852613
         Iterations: 183
         Function evaluations: 369
Optimization terminated successfully.
         Current function value: 1459.417046
         Iterations: 167
         Function evaluations: 332
Optimization terminated successfully.
         Current function value: 1308.981458
         Iterations: 157
         Function evaluations: 311
Optimization terminated successfully.
         Current function value: 1158.545788
         Iterations: 152
         Function evaluations: 299
Optimization terminated successfully.
         Current function value: 1008.109804
         Iterations: 141
         Function evaluations: 268

In [19]:
theta_guess = [0.3, 0.3]
params_quad = [optimize.fmin(chi2_from_likelihood_quad, theta_guess, args=(E_QG, del_t_obs, z, sigma_del_t, K_quad), maxfun=5000) for E_QG in x]
print("Length of params n=2 : ", len(params_quad))

Optimization terminated successfully.
         Current function value: -185.955776
         Iterations: 45
         Function evaluations: 88
Optimization terminated successfully.
         Current function value: -274.787698
         Iterations: 39
         Function evaluations: 76
Optimization terminated successfully.
         Current function value: -278.325189
         Iterations: 38
         Function evaluations: 75
Optimization terminated successfully.
         Current function value: -278.472132
         Iterations: 39
         Function evaluations: 77
Optimization terminated successfully.
         Current function value: -278.481655
         Iterations: 39
         Function evaluations: 76
Optimization terminated successfully.
         Current function value: -278.482286
         Iterations: 38
         Function evaluations: 74
Optimization terminated successfully.
         Current function value: -278.482329
         Iterations: 38
         Function evaluations: 74
Optimization 

In [20]:
for E_QG in x:
    chi2_n1_values = [chi2_from_likelihood(ps, E_QG, del_t_obs, z, sigma_del_t, K_linear) for ps in params_linear]

chi2min_n1 = min(chi2_n1_values)

print(chi2_n1_values-chi2min_n1)

[5.00852343e+03 4.85808801e+03 4.70765245e+03 4.55721738e+03
 4.40678315e+03 4.25635318e+03 4.10593892e+03 3.95558546e+03
 3.80546242e+03 3.65622029e+03 3.51023085e+03 3.37517529e+03
 3.25482139e+03 2.91319332e+03 1.95960269e+03 7.85234079e+02
 1.11931644e+02 8.71641543e+00 6.10047500e-01 4.23094273e-02
 3.48717533e-03 1.76095483e-04 2.30335738e-05 0.00000000e+00
 0.00000000e+00]


In [21]:
for E_QG in x:
    chi2_n2_values = [chi2_from_likelihood_quad(ps, E_QG, del_t_obs, z, sigma_del_t, K_quad) for ps in params_quad]

chi2min_n2 = min(chi2_n2_values)

print (chi2_n2_values-chi2min_n2)

[8.49277722e+02 3.41571683e+01 1.82567164e-01 6.93230002e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00]


In [22]:
#plt.figure()
#ax=plt.axes([0.17,0.17,0.8,0.8])
plt.loglog(x,chi2_n1_values-chi2min_n1,'k-*',lw=2)
plt.xlabel('$E_{QG}$ (GeV)',fontsize=14,fontweight='bold')
plt.ylabel('$\Delta \chi^2$',fontsize=14,fontweight='bold')
plt.legend(['n=1 LIV'],fontsize=14,frameon=False)
#plt.tick_params(labelsize=18)
plt.axhline(y=4.0,color='magenta',lw=2,linestyle='--')
#plt.axhline(x=1.0e+17,color='magenta',lw=2,linestyle='--')
#plt.gca().set_ylim(bottom=5.0e-05)
plt.vlines(x=1.22326e+15,ymin=1.0e-05,ymax=4.0,linestyles='dotted',color='magenta',lw=2)
plt.xlim(1e5, 1e19)
plt.grid(True)
plt.show()
#plt.savefig('linearLIV_95cl.pdf')

<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_6688/3208720070.py:5: SyntaxWarning: invalid escape sequence '\D'
  plt.ylabel('$\Delta \chi^2$',fontsize=14,fontweight='bold')


In [23]:
#plt.figure()
#ax=plt.axes([0.17,0.17,0.8,0.8])
plt.loglog(x,chi2_n2_values-chi2min_n2,'k-*',lw=2)
plt.xlabel('$E_{QG}$ (GeV)',fontsize=14,fontweight='bold')
plt.ylabel('$\Delta \chi^2$',fontsize=14,fontweight='bold')
plt.legend(['n=2 LIV'],fontsize=14,frameon=False)
#plt.tick_params(labelsize=18)
plt.axhline(y=4.0,color='magenta',lw=2,linestyle='--')
#plt.axhline(x=1.0e+17,color='magenta',lw=2,linestyle='--')
#plt.gca().set_ylim(bottom=5.0e-05)
plt.vlines(x=6.64460e+5,ymin=1.0e-05,ymax=4.0,linestyles='dotted',color='magenta',lw=2)
plt.xlim(1e5, 1e10)
plt.grid(True)
plt.show()
#plt.savefig('quadLIV_95cl.pdf')

<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_6688/2423382261.py:5: SyntaxWarning: invalid escape sequence '\D'
  plt.ylabel('$\Delta \chi^2$',fontsize=14,fontweight='bold')


## Minimizing chi2 with the 95% cl lower limit values of E_{QG}

In [24]:
Eqg_95cl_n1 = 1.22326e+15
Eqg_95cl_n2 = 6.64460e+5

In [25]:
def chi2_from_eq9(theta, E_qg, del_T, z, sigma_del_T, k):
    b, sigma_int = theta
    a_liv = E_0/(1.0*H_0*E_qg)
    model = a_liv*k + b
    term_num = ((del_T/(1+z)) - (model))**2
    term_den = (sigma_int**2) + ((sigma_del_T/(1+z))**2)
    #term1 = -2*np.log(1/np.sqrt(sigma_int**2 + ((sigma_del_T/(1+z))**2)))
    term2 = term_num/term_den
    return np.sum(term2)

In [26]:
def chi2_from_eq9_quad(theta, E_qg, del_T, z, sigma_del_T, k):
    b, sigma_int = theta
    a_liv = (3.0*E_0_quad)/(2.0*H_0*(E_qg**2))
    model = a_liv*k + b
    term_num = ((del_T/(1+z)) - (model))**2
    term_den = (sigma_int**2) + ((sigma_del_T/(1+z))**2)
    #term1 = -2*np.log(1/np.sqrt(sigma_int**2 + ((sigma_del_T/(1+z))**2)))
    term2 = term_num/term_den
    return np.sum(term2)

In [27]:
theta_guess = [0.03, 0.03]
params_linear_opt = optimize.fmin(chi2_from_likelihood, theta_guess, args=(Eqg_95cl_n1, del_t_obs, z, sigma_del_t, K_linear), maxfun=5000)
print("b, sigma_int : ", params_linear_opt)
print(r"$\chi^2$/DOF -- from eq9 = ", chi2_from_eq9(params_linear_opt, Eqg_95cl_n1, del_t_obs, z, sigma_del_t, K_linear), "/ 54 = ", chi2_from_eq9(params_linear_opt, Eqg_95cl_n1, del_t_obs, z, sigma_del_t, K_linear)/(56-2))

Optimization terminated successfully.
         Current function value: -278.217697
         Iterations: 27
         Function evaluations: 53
b, sigma_int :  [0.0096268  0.02033102]
$\chi^2$/DOF -- from eq9 =  53.68669768274461 / 54 =  0.9941981052360113


In [28]:
theta_guess = [0.3, 0.3]
params_quad_opt = optimize.fmin(chi2_from_likelihood_quad, theta_guess, args=(Eqg_95cl_n2, del_t_obs, z, sigma_del_t, K_quad), maxfun=5000)
print("b, sigma_int : ", params_quad_opt)
print(r"$\chi^2$/DOF -- from eq9 = ", chi2_from_eq9_quad(params_quad_opt, Eqg_95cl_n2, del_t_obs, z, sigma_del_t, K_quad), "/ 54 = ", chi2_from_eq9_quad(params_quad_opt, Eqg_95cl_n2, del_t_obs, z, sigma_del_t, K_quad)/(56-2))

Optimization terminated successfully.
         Current function value: -277.572544
         Iterations: 38
         Function evaluations: 76
b, sigma_int :  [ 0.00932272 -0.02085184]
$\chi^2$/DOF -- from eq9 =  53.33185530821507 / 54 =  0.9876269501521309


## Minimizing chi2 at the Planck energy

In [29]:
theta_guess = [0.03, 0.03]
params_linear_pl = optimize.fmin(chi2_from_likelihood, theta_guess, args=(1e19, del_t_obs, z, sigma_del_t, K_linear), maxfun=5000)
print("b, sigma_int : ", params_linear_pl)
print(r"$\chi^2$/DOF -- from eq9 = ", chi2_from_eq9(params_linear_pl, 1e19, del_t_obs, z, sigma_del_t, K_linear), "/ 54 = ", chi2_from_eq9(params_linear_pl, 1e19, del_t_obs, z, sigma_del_t, K_linear)/(56-2))

Optimization terminated successfully.
         Current function value: -278.482332
         Iterations: 24
         Function evaluations: 43
b, sigma_int :  [0.01875479 0.0200804 ]
$\chi^2$/DOF -- from eq9 =  53.90573837118531 / 54 =  0.9982544142812094


In [30]:
theta_guess = [0.3, 0.3]
params_quad_pl = optimize.fmin(chi2_from_likelihood_quad, theta_guess, args=(1e19, del_t_obs, z, sigma_del_t, K_quad), maxfun=5000)
print("b, sigma_int : ", params_quad_pl)
print(r"$\chi^2$/DOF -- from eq9 = ", chi2_from_eq9_quad(params_quad_pl, 1e19, del_t_obs, z, sigma_del_t, K_quad), "/ 54 = ", chi2_from_eq9_quad(params_quad_pl, 1e19, del_t_obs, z, sigma_del_t, K_quad)/(56-2))

Optimization terminated successfully.
         Current function value: -278.482332
         Iterations: 38
         Function evaluations: 74
b, sigma_int :  [0.01872501 0.02005716]
$\chi^2$/DOF -- from eq9 =  53.95066648465478 / 54 =  0.999086416382496


## Plotting dependence of the rescaled time lag on K(z)

In [31]:
def best_fit_linear(b, Eqg):
    a_liv = E_0/(1.0*H_0*Eqg)
    x = np.linspace(0.12, 0.28, 1000)
    return a_liv*x + b

In [32]:
plt.title('Rescaled time lag dependence on K(z)-linear')
plt.xlabel('K(z)')
plt.ylabel('Rescaled spectral time lag')
plt.errorbar(K_linear, rescaled_del_t_obs, yerr=sigma_del_t, fmt='rx', capsize=0, markersize=4, linestyle='None')
plt.plot(np.linspace(0.12, 0.28, 1000), best_fit_linear(params_linear_opt[0], Eqg_95cl_n1), color='blue')
plt.grid(True)
plt.tight_layout()
plt.show()

## Testing minimizers for n=2

In [ ]:
theta_guess = [0.3, 0.3]
params = [
    optimize.minimize(
        chi2_from_likelihood_quad, 
        theta_guess, 
        args=(E_QG, del_t_obs, z, sigma_del_t, K_quad), 
        method="Powell",  # Change this to any method you want (e.g., "BFGS", "Powell", "L-BFGS-B")
        options={"maxiter": 20000, "disp": True}
    ) 
    for E_QG in x
]

params = [obj.x for obj in params]